In [55]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import urllib.parse
import os

In [56]:
# Define a request header (to prevent anti-scraping)
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [57]:
url='https://arxiv.org/list/cs/pastweek?skip=0&show=100'
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [59]:
title = soup.select('div.list-title.mathjax')
subject = soup.select('div.list-subjects')
author = soup.select('div.list-authors')
abstract_urls = soup.select('span.list-identifier')

url_abs=[urllib.parse.urljoin(url,abstract_url.a["href"]) for abstract_url in abstract_urls]
abs = []
for i in range(len(url_abs)):
    page_abs = requests.get(url_abs[i], headers=headers)
    soup_abs = BeautifulSoup(page_abs.content, 'html.parser')
    abstract = soup_abs.select('blockquote.abstract.mathjax')
    abs.extend(abstract)

In [62]:
title=[t.text[8:] for t in title]
subject=[s.text[11:] for s in subject]
author=[''.join(a.text[9:].split('\n')) for a in author]
abstracts=[a.text[11:] for a in abs]

csv_name =  "100_cs_papers.csv"
csv_dir = "./output"
movie_df = pd.DataFrame({"title": title, "subject": subject, "author": author, "abstract": abstracts})
movie_df.head()
os.makedirs(csv_dir, exist_ok=True)
movie_df.to_csv(csv_dir + '/' + csv_name, index=False)